In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.08 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.66 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.05 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00   7.32 MB/s
vincent-0.4.4- 100% |###################

In [2]:
CLIENT_ID = 'J5RGS1TUKKZZQS21FHVCN55GWRA33WOY5WIMQCDCJGBV2YZK' # your Foursquare ID
CLIENT_SECRET = 'IDI3JJ5ZQD0T1L3QVNIUYA420ETNXM0FSBFALKYIYKCRXY0F' # your Foursquare Secret
VERSION = '20181112' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J5RGS1TUKKZZQS21FHVCN55GWRA33WOY5WIMQCDCJGBV2YZK
CLIENT_SECRET:IDI3JJ5ZQD0T1L3QVNIUYA420ETNXM0FSBFALKYIYKCRXY0F


In [3]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [6]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&sortByDistance=True'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

column_names = ['Name', 'Latitude', 'Longitude', 'Distance', 'CategoryName'] 

datafinal = pd.DataFrame(columns=column_names)

venues = results['response']['groups'][0]['items']
for venue in venues:
    name = venue['venue']['name'] #name
    lat = venue['venue']['location']['lat'] #latitude
    long = venue['venue']['location']['lng'] #lng
    distance = venue['venue']['location']['distance'] # distance
    categoryname = venue['venue']['categories'][0]['name'] #category name
    datafinal = datafinal.append({'Name': name,
                              'Latitude': lat,
                              'Longitude': long,
                              'Distance': distance,
                              'CategoryName': categoryname}, ignore_index=True)
datafinal.head()

,Name,Latitude,Longitude,Distance,CategoryName
0,Central Park Tennis Center,40.789313,-73.961862,191,Tennis Court
1,North Meadow,40.792027,-73.959853,215,Park
2,East Meadow,40.790160,-73.955498,365,Park
3,Central Park - 96th Street Playground,40.787845,-73.956369,383,Playground
4,Rudin Family Playground,40.791822,-73.964147,411,Playground


In [7]:
#we make the map of manhattan
map_mh = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, categoryname in zip(datafinal['Latitude'], datafinal['Longitude'], datafinal['CategoryName']):
    label = '{}'.format(categoryname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mh)  
    
map_mh

In [10]:
#we group by category and get the %
categorynamedf = datafinal[['CategoryName']]
categorynamedf['Number of Occurences'] = 1
categorynamedf = categorynamedf.groupby('CategoryName').count()
categorynamedf['Percentage of Occurences'] = categorynamedf['Number of Occurences']/(categorynamedf['Number of Occurences'].sum())
categorynamedf = categorynamedf.sort_values(by='Percentage of Occurences', ascending=False)
categorynamedf.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Number of Occurences,Percentage of Occurences
CategoryName,,
Park,9,0.09
Playground,6,0.06
Scenic Lookout,5,0.05
Wine Shop,4,0.04
Plaza,4,0.04
